In [1]:
! pip install transformers[sentencepiece] datasets sacrebleu rouge_score py7zr -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 2.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.4/487.4 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.9/67.9 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 58.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.2/96.2 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.6/50.6 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 84.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.3/141.3 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 413.7/413.7 kB 20.2 MB/s eta 0:00:00
   ━━━━━━

In [5]:
! pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 3.2 MB/s eta 0:00:00


# summarization baseline

# text summarization pipeline

In [34]:
from datasets import load_dataset
import nltk
from nltk.tokenize import sent_tokenize

nltk.download('punkt_tab')

dataset = load_dataset("cnn_dailymail", "3.0.0")
sample_text = dataset['train'][1]['article'][:1000]

def baseline_summary_three_sent(text):
    sentences = sent_tokenize(text)
    return '\n'.join(sentences[:3])  # First 3 sentences

summaries = {}
summaries['baseline'] = baseline_summary_three_sent(sample_text)

print(summaries['baseline'])  # Check output


[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


Editor's note: In our Behind the Scenes series, CNN correspondents share their experiences in covering news and analyze the stories behind the events.
Here, Soledad O'Brien takes users inside a jail where many of the inmates are mentally ill. An inmate housed on the "forgotten floor," where many mentally ill inmates are housed in Miami before trial.
MIAMI, Florida (CNN) -- The ninth floor of the Miami-Dade pretrial detention facility is dubbed the "forgotten floor."


In [35]:
from transformers import pipeline, set_seed
set_seed(42)
pipe = pipeline('text-generation', model='gpt2-medium')
gpt2_query = sample_text + '\nTL;DR:\n'
pipe_out = pipe(gpt2_query, max_length=512, clean_up_tokenization_spaces=True)

config.json:   0%|          | 0.00/718 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.52G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Device set to use cuda:0
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [36]:
pipe_out

[{'generated_text': 'Editor\'s note: In our Behind the Scenes series, CNN correspondents share their experiences in covering news and analyze the stories behind the events. Here, Soledad O\'Brien takes users inside a jail where many of the inmates are mentally ill. An inmate housed on the "forgotten floor," where many mentally ill inmates are housed in Miami before trial. MIAMI, Florida (CNN) -- The ninth floor of the Miami-Dade pretrial detention facility is dubbed the "forgotten floor." Here, inmates with the most severe mental illnesses are incarcerated until they\'re ready to appear in court. Most often, they face drug charges or charges of assaulting an officer --charges that Judge Steven Leifman says are usually "avoidable felonies." He says the arrests often result from confrontations with police. Mentally ill people often won\'t do what they\'re told when police arrive on the scene -- confrontation seems to exacerbate their illness and they become more paranoid, delusional, and

In [38]:
pipe_out[0]['generated_text'][len(gpt2_query):]

'- The 10th-floor jail, called "the secret prison," was developed specifically for criminally insane people who cannot make bond.\n- It\'s one of only four jails in Miami that houses mentally ill people. The jail was built to house mentally ill people who had "unpredictable behavior"\nTL;DR; The tenth-floor jail was designed to be the secret prison in the world\'s most dangerous jail and has become one where almost all of the mentally ill are housed.\nA mentally ill man, David Garcia-Lopez, has been locked up there for nearly a decade, but he still has his cell phone, despite his wishes. The 10th floor jail has been open for years, but a new administration began installing new surveillance equipment in January 2010. The surveillance cameras are installed above and below ground, a security protocol designed to keep jailers from watching each cell and suspect entering and exiting from another cell. The facility is one of the more violent in Miami. During his five-month incarceration, Gar

In [40]:
summaries['gpt2'] = '\n'.join(sent_tokenize(pipe_out[0]['generated_text'][len(gpt2_query):]))
summaries['gpt2']

'- The 10th-floor jail, called "the secret prison," was developed specifically for criminally insane people who cannot make bond.\n- It\'s one of only four jails in Miami that houses mentally ill people.\nThe jail was built to house mentally ill people who had "unpredictable behavior"\nTL;DR; The tenth-floor jail was designed to be the secret prison in the world\'s most dangerous jail and has become one where almost all of the mentally ill are housed.\nA mentally ill man, David Garcia-Lopez, has been locked up there for nearly a decade, but he still has his cell phone, despite his wishes.\nThe 10th floor jail has been open for years, but a new administration began installing new surveillance equipment in January 2010.\nThe surveillance cameras are installed above and below ground, a security protocol designed to keep jailers from watching each cell and suspect entering and exiting from another cell.\nThe facility is one of the more violent in Miami.\nDuring his five-month incarceration

# T5

In [41]:
pipe = pipeline('summarization', model='t5-small')
pipe_out = pipe(sample_text)

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

Device set to use cuda:0


In [42]:
pipe_out

[{'summary_text': "inmates with the most severe mental illnesses are incarcerated until they're ready to appear in court . most often, they face drug charges or charges of assaulting an officer . mentally ill people become more paranoid, delusional, and less likely to follow dir ."}]

In [43]:
summaries['t5'] = '\n'.join(sent_tokenize(pipe_out[0]['summary_text']))
summaries['t5']

"inmates with the most severe mental illnesses are incarcerated until they're ready to appear in court .\nmost often, they face drug charges or charges of assaulting an officer .\nmentally ill people become more paranoid, delusional, and less likely to follow dir ."

# BART

In [44]:
pipe = pipeline('summarization', model='facebook/bart-large-cnn')
pipe_out = pipe(sample_text)

config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Device set to use cuda:0


In [45]:
pipe_out

[{'summary_text': 'Miami-Dade pretrial detention facility is dubbed the "forgotten floor" Here, inmates with the most severe mental illnesses are incarcerated. Most often, they face drug charges or charges of assaulting an officer. Judge Steven Leifman says the arrests often result from confrontations with police.'}]

In [46]:
summaries['bart'] = '\n'.join(sent_tokenize(pipe_out[0]['summary_text']))
summaries['bart']

'Miami-Dade pretrial detention facility is dubbed the "forgotten floor" Here, inmates with the most severe mental illnesses are incarcerated.\nMost often, they face drug charges or charges of assaulting an officer.\nJudge Steven Leifman says the arrests often result from confrontations with police.'

# PEGASUS

In [48]:
pipe = pipeline('summarization', model='google/pegasus-cnn_dailymail')
pipe_out = pipe(sample_text)

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Device set to use cuda:0


In [49]:
pipe_out

[{'summary_text': 'Mentally ill inmates are housed on the "forgotten floor" of a Miami jail .<n>Judge Steven Leifman says the charges are usually "avoidable felonies"<n>He says the arrests often result from confrontations with police .<n>Mentally ill people often won\'t do what they\'re told when police arrive on the scene .'}]

In [50]:
summaries['pegasus'] = pipe_out[0]['summary_text'].replace(' .<n>', '.\n')
summaries['pegasus']

'Mentally ill inmates are housed on the "forgotten floor" of a Miami jail.\nJudge Steven Leifman says the charges are usually "avoidable felonies"<n>He says the arrests often result from confrontations with police.\nMentally ill people often won\'t do what they\'re told when police arrive on the scene .'

# comparing different summaries

In [51]:
print('GROUND TRUTH')
print(dataset['train'][1]['highlights'])

for model_name in summaries:
    print(f'\n{model_name.upper()}')
    print(summaries[model_name])

GROUND TRUTH
Mentally ill inmates in Miami are housed on the "forgotten floor"
Judge Steven Leifman says most are there as a result of "avoidable felonies"
While CNN tours facility, patient shouts: "I am the son of the president"
Leifman says the system is unjust and he's fighting for change .

BASELINE
Editor's note: In our Behind the Scenes series, CNN correspondents share their experiences in covering news and analyze the stories behind the events.
Here, Soledad O'Brien takes users inside a jail where many of the inmates are mentally ill. An inmate housed on the "forgotten floor," where many mentally ill inmates are housed in Miami before trial.
MIAMI, Florida (CNN) -- The ninth floor of the Miami-Dade pretrial detention facility is dubbed the "forgotten floor."

GPT2
- The 10th-floor jail, called "the secret prison," was developed specifically for criminally insane people who cannot make bond.
- It's one of only four jails in Miami that houses mentally ill people.
The jail was buil

# SacreBLEU

In [55]:

bleu_metric = load_metric('sacrebleu')

In [67]:
import numpy as np
import pandas as pd
from evaluate import load as load_metric

# Load BLEU metric
bleu_metric = load_metric("bleu")

# Ensure predictions and references are properly structured
predictions = [summaries['pegasus']]
references = [[dataset['train'][1]['highlights']]]

# Compute BLEU score
results = bleu_metric.compute(predictions=predictions, references=references)

# Round precision values for readability
results['precisions'] = [np.round(p, 2) for p in results['precisions']]

# Display as DataFrame
pd.DataFrame.from_dict(results, orient='index', columns=['Value'])


,Value
bleu,0.248734
precisions,"[0.5, 0.28, 0.21, 0.13]"
brevity_penalty,1.0
length_ratio,1.017544
translation_length,58
reference_length,57


# ROUGE

In [68]:
rouge_metric = load_metric('rouge')

In [70]:
rouge_names = ['rouge1', 'rouge2', 'rougeL', 'rougeLsum']

reference = dataset['train'][1]['highlights']

records = []

for model_name in summaries:
    rouge_metric.add(prediction=summaries[model_name], reference=reference)
    score = rouge_metric.compute()
    rouge_dict = {rn: score[rn] for rn in rouge_names}

    records.append(rouge_dict)

pd.DataFrame.from_records(records, index=summaries.keys())

,rouge1,rouge2,rougeL,rougeLsum
baseline,0.365079,0.145161,0.206349,0.285714
gpt2,0.165563,0.026667,0.099338,0.165563
t5,0.197802,0.022472,0.131868,0.197802
bart,0.365591,0.131868,0.215054,0.322581
pegasus,0.500000,0.244898,0.360000,0.460000


# Evaluating on test set of the CNN/DailyMail dataset

In [71]:
def calculate_metric_on_baseline_test_ds(dataset, metric, column_text='article', column_summary='highlights'):
    summaries = [baseline_summary_three_sent(text) for text in dataset[column_text]]
    metric.add_batch(predictions=summaries, references=dataset[column_summary])
    score = metric.compute()
    return score

In [72]:
test_sampled = dataset['train'].shuffle(seed=42).select(range(1000))
score = calculate_metric_on_baseline_test_ds(test_sampled, rouge_metric)
rouge_dict = {rn: score[rn] for rn in rouge_names}
pd.DataFrame.from_dict(rouge_dict, orient='index', columns=['baseline rouge'])

,baseline rouge
rouge1,0.253995
rouge2,0.100642
rougeL,0.165754
rougeLsum,0.231571


# strategy to calculate the rouge metric on test dataset for the other models

In [76]:
from tqdm import tqdm
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"

def chunks(list_of_elements, batch_size):
    for i in range(0, len(list_of_elements), batch_size):
        yield list_of_elements[i : i + batch_size]

def calculate_metric_on_test_ds(dataset, metric, model, tokenizer, batch_size=1, device=device, column_text='article', column_summary='highlights'):
    article_batches = list(chunks(dataset[column_text], batch_size))
    target_batches = list(chunks(dataset[column_summary], batch_size))

    for article_batch, target_batch in tqdm(zip(article_batches, target_batches), total=len(article_batches)):
        inputs = tokenizer(article_batch, max_length=1024, truncation=True, padding='max_length', return_tensors='pt')
        summaries = model.generate(input_ids=inputs['input_ids'].to(device),
                                   attention_mask=inputs['attention_mask'].to(device),
                                   length_penalty=0.8,
                                   num_beams=8,
                                   max_length=128)
        decoded_summaries = [tokenizer.decode(s, skip_special_tokens=True, clean_up_tokenization_spaces=True) for s in summaries]

        # Correct replacement of <pad> tokens
        decoded_summaries = [d.replace("<pad>", " ") for d in decoded_summaries]

        metric.add_batch(predictions=decoded_summaries, references=target_batch)

    score = metric.compute()
    return score


In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

model_ckpt = 'google/pegasus-cnn_dailymail'
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
model = AutoModelForSeq2SeqLM.from_pretrained(model_ckpt).to(device)

batch_size = 1  # Define batch size
score = calculate_metric_on_test_ds(test_sampled, rouge_metric, model, tokenizer, batch_size)

# Assuming calculate_metric_on_test_ds returns rouge_dict
pd.DataFrame(score, index=['pegasus'])


Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
  2%|▏         | 20/1000 [00:45<38:50,  2.38s/it]